In [4]:
import re
import os

def get_outcomes(filename):
    text = open(filename, 'r', encoding='utf-8').read()

    # Split the text based on the team name

    halfs = re.split(r'herItem_TabText ">Full Time Score</div></div><div data-con', text)

    text = halfs[1]

    split_text = re.split(r'<div class="bbl-StickyMarketColumnHeader_Label ">(.*?)<\/div>', text)

    home, draw, away = split_text.pop(1), split_text.pop(2), split_text.pop(3)

    game = f"{home} vs {away}"

    team = [home, draw, away]

    odds_dict = {}

    # Print the split text
    for i, item in enumerate(split_text[1:]):

        # Extract scores, teams, and odds using regular expressions
        pattern = r'<span class="bbl-BetBuilderParticipant_Name">(.*?)<\/span><span class="bbl-BetBuilderParticipant_Odds">(.*?)<\/span>'
        matches = re.findall(pattern, item)

        # Create a table of teams, scores, and odds
        table = []
        for match in matches:
            score = match[0]
            odds = match[1]
            numer, denom = odds.split('/')
            # Specify the third value here
            decimal_odds = 1 + int(numer) / int(denom)  # Replace 'Some value' with your desired value
            table.append((game, score, odds, decimal_odds))

        # Print the table
        # for row in table:
        #     print(row)

        odds_dict[team[i]] = table


    #print(odds_dict)

    # Find the team with the highest odds
    outcomes = []
    for team, table in odds_dict.items():
        for row in table:
            game, score, odds, decimal_odds = row
            outcome = (game, team, score, odds, decimal_odds)
            outcomes.append(outcome)

    # Sort the outcomes list in ascending order based on the fourth item in each tuple
    outcomes = sorted(outcomes, key=lambda x: x[4])

    # Print all the outcomes
    return outcomes

full_outcomes = get_outcomes('full.txt')

In [13]:
from Scrapper import get_outcomes

import os

directory = r'C:\Users\adamf\PycharmProjects\Visual Studio\Bet365_scrapper\Matches 060224'
best = []
second_best = []
third_best = []


for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        outcomes = get_outcomes(directory + '/' + filename)
        best.append(outcomes[0])
        second_best.append(outcomes[1])
        third_best.append(outcomes[2])
        continue
    else:
        continue

top_3 =best + second_best + third_best
top_3 = sorted(top_3, key=lambda x: x[0])
top_3

[('Fulham vs Bournemouth', 'Draw', '1-1', '13/2', 7.5),
 ('Fulham vs Bournemouth', 'Fulham', '2-1', '9/1', 10.0),
 ('Fulham vs Bournemouth', 'Fulham', '1-0', '10/1', 11.0),
 ('Liverpool vs Burnley', 'Liverpool', '2-0', '15/2', 8.5),
 ('Liverpool vs Burnley', 'Liverpool', '3-0', '15/2', 8.5),
 ('Liverpool vs Burnley', 'Liverpool', '3-1', '10/1', 11.0),
 ('Luton vs Sheff Utd', 'Draw', '1-1', '7/1', 8.0),
 ('Luton vs Sheff Utd', 'Luton', '1-0', '8/1', 9.0),
 ('Luton vs Sheff Utd', 'Luton', '2-1', '8/1', 9.0),
 ('Nottm Forest vs Newcastle', 'Draw', '1-1', '7/1', 8.0),
 ('Nottm Forest vs Newcastle', 'Newcastle', '2-1', '17/2', 9.5),
 ('Nottm Forest vs Newcastle', 'Newcastle', '1-0', '9/1', 10.0),
 ('Tottenham vs Brighton', 'Draw', '1-1', '17/2', 9.5),
 ('Tottenham vs Brighton', 'Tottenham', '2-1', '9/1', 10.0),
 ('Tottenham vs Brighton', 'Draw', '2-2', '11/1', 12.0),
 ('Wolverhampton vs Brentford', 'Draw', '1-1', '13/2', 7.5),
 ('Wolverhampton vs Brentford', 'Wolverhampton', '1-0', '9/1', 1

In [25]:
import pandas as pd
best_df = pd.DataFrame(best, columns=['Game', 'Team', 'Score', 'Odds', 'Decimal Odds'])
best_df['Probability'] = 1 / best_df['Decimal Odds']
best_df

,Game,Team,Score,Odds,Decimal Odds,Probability
0,Fulham vs Bournemouth,Draw,1-1,13/2,7.5,0.133333
1,Liverpool vs Burnley,Liverpool,2-0,15/2,8.5,0.117647
2,Luton vs Sheff Utd,Draw,1-1,7/1,8.0,0.125000
3,Nottm Forest vs Newcastle,Draw,1-1,7/1,8.0,0.125000
4,Tottenham vs Brighton,Draw,1-1,17/2,9.5,0.105263
5,Wolverhampton vs Brentford,Draw,1-1,13/2,7.5,0.133333


In [26]:
second_best_df = pd.DataFrame(second_best, columns=['Game', 'Team', 'Score', 'Odds', 'Decimal Odds'])
second_best_df['probability'] = 1/second_best_df['Decimal Odds']
second_best_df.sort_values(by='probability', ascending=False)
second_best_df

,Game,Team,Score,Odds,Decimal Odds,probability
0,Fulham vs Bournemouth,Fulham,2-1,9/1,10.0,0.100000
1,Liverpool vs Burnley,Liverpool,3-0,15/2,8.5,0.117647
2,Luton vs Sheff Utd,Luton,1-0,8/1,9.0,0.111111
3,Nottm Forest vs Newcastle,Newcastle,2-1,17/2,9.5,0.105263
4,Tottenham vs Brighton,Tottenham,2-1,9/1,10.0,0.100000
5,Wolverhampton vs Brentford,Wolverhampton,1-0,9/1,10.0,0.100000


In [19]:
df = pd.DataFrame(top_3, columns = ['Game', 'Team', 'Score', 'Odds', 'Decimal Odds'])
df['probability'] = 1/df['Decimal Odds']


,Game,Team,Score,Odds,Decimal Odds,probability
0,Fulham vs Bournemouth,Draw,1-1,13/2,7.5,0.133333
1,Fulham vs Bournemouth,Fulham,2-1,9/1,10.0,0.100000
2,Fulham vs Bournemouth,Fulham,1-0,10/1,11.0,0.090909
3,Liverpool vs Burnley,Liverpool,2-0,15/2,8.5,0.117647
4,Liverpool vs Burnley,Liverpool,3-0,15/2,8.5,0.117647
5,Liverpool vs Burnley,Liverpool,3-1,10/1,11.0,0.090909
6,Luton vs Sheff Utd,Draw,1-1,7/1,8.0,0.125000
7,Luton vs Sheff Utd,Luton,1-0,8/1,9.0,0.111111
8,Luton vs Sheff Utd,Luton,2-1,8/1,9.0,0.111111
9,Nottm Forest vs Newcastle,Draw,1-1,7/1,8.0,0.125000


In [30]:
results = []

probabilities = df['probability'].tolist()
score=1

for i in range(0, len(probabilities), 3):
    score *= probabilities[i]
    results.append(score)
    
print(score)

2.3194938407434883e-05


In [ ]:
selections_mask = [[1, 0, 0],
                   [1, 0, 0],
                   [1, 0, 0],
                   [1, 0, 0],
                   [1, 0, 0],
                   [1, 0, 0],


                   ]

216